# 実験実行環境を再作成した場合のセットアップを行う

研究リポジトリの実行環境タブから実験実行環境を作成した場合に必要な初期セットアップを行います。<br>
このタスクを実行すると、実験データの更新や実験フローの利用が可能になります。<br>
上から順番に実行してください。

Jupyter Notebookの操作方法については、[こちら](https://support.rdm.nii.ac.jp/usermanual/60/#jp_operation)をご覧ください。新規タブで開きます。

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、当実験実行環境で操作する研究リポジトリ名を確認できます。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('blue')

## 1. ユーザー認証を行う

この手順では、GIN-forkと同期するための準備として、あなたのユーザー情報をシステムに認証させる手続きを行います。<br>
以下のセルを実行し、画面の表示に沿ってGIN-forkに登録したユーザー名、パスワード、メールアドレスを入力してください。<br>
各項目入力後、『入力を完了する』ボタンを押下してください。<br>

入力値が正しくユーザ認証が通った場合、ボタンが緑に変化します。<br>

『入力を完了する』ボタンを押下後、入力値が正しくない場合ボタンが黄色に変化します。<br>
ボタンに表示される指示に従い、再入力後、ボタンを押下してください。<br> 

In [ ]:
import os
path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils
# ユーザ認証
utils.initial_gin_user_auth()

## 2. データ同期のための設定をする

この手順では、今の実行環境とGIN-forkのリポジトリでデータの同期をとるための準備をします。  
以下のセルを実行してください。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus
import json
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util
from utils.token import token

os.chdir(path_flow_root)
from util.scripts import utils

params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)
pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'/api/v1/user/keys?token=' + token.get_ginfork_token(),
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

if response.status_code == HTTPStatus.CREATED:
    display_util.display_info('公開鍵の用意が出来ました。')
elif msg['message'] == 'Key content has been used as non-deploy key':
    display_util.display_warm('すでに公開鍵の用意が完了しています。')

In [ ]:
import os
import subprocess
from datalad import api
from IPython.display import clear_output
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util
from utils.common import common

# sibling url をsshに変更する
os.chdir(os.environ['HOME'])
http_url = subprocess.getoutput('git config --get remote.origin.url')
http_url, build_token = common.convert_url_remove_user_token(http_url)
ssh_url = http_url.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])

# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
display_util.display_info('SSH接続の準備が完了しました。')

In [ ]:
import os
os.chdir(path_flow_root)
from util.scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

# S3にあるデータをGIN-forkに同期しないための設定
!git annex untrust here
!git annex --force trust web

## 3. 当実行環境で編集する実験パッケージを選択する

以下のセルを実行して、当実験実行環境で編集したい実験パッケージを1つ選択してください。  
選択を間違えた場合は、再度このセルから実行して、選択しなおしてください。

In [ ]:
from ipywidgets import Dropdown, Button
from IPython.display import clear_output
import os
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util

input_repo_experiments_path = '/home/jovyan/experiments'

ex_pkg_list = list[str]()
if os.path.isdir(input_repo_experiments_path):
    for data_name in os.listdir(input_repo_experiments_path):
        data_path = os.path.join(input_repo_experiments_path,data_name)
        if os.path.isdir(data_path):
            ex_pkg_list.append(data_name)

if len(ex_pkg_list) > 0:
    def on_click_callback(clicked_button: Button) -> None:
        global experiment_title
        experiment_title=dropdown.value
        clear_output()
        display_util.display_info("入力を受けつけました。")
        display_util.display_msg("実験パッケージ名: "+ experiment_title)

    dropdown = Dropdown(
        options=ex_pkg_list,
        description='実験パッケージ名:',
        disabled=False,
        style= {'description_width': 'initial'}
    )

    button = Button(description='入力完了')
    button.on_click(on_click_callback)
    display_util.display_info("実験パッケージの選択後、入力完了ボタンを押下してください。")
    display(dropdown, button)
else:
    display_util.display_warm('このリポジトリでは実験パッケージが存在しません。')

## 4. 実験パッケージのデータを用意する

選択した実験パッケージのデータを当実験実行環境に用意します。

In [ ]:
import os
from IPython.display import clear_output
os.chdir(os.environ['HOME'] + '/WORKFLOWS')
from utils import display_util

EXPERIMENT_TITLE = experiment_title
%store EXPERIMENT_TITLE
clear_output()
os.chdir(os.environ['HOME'] + '/experiments/' + experiment_title)
try:
    !datalad get .
    !datalad unlock .
except Exception as e:
    display_util.display_err('データの用意に失敗しました。')
    display_util.display_log(str(e))
else:
    clear_output()
    display_util.display_info('データの用意が完了しました。次にお進みください。')

## 5. 実行環境一覧を更新する
GIN-forkの実行環境一覧に当実行環境を追加します。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

%store -r EXPERIMENT_TITLE
# 実行環境を追加
utils.add_container(experiment_title=EXPERIMENT_TITLE)

## 6. 実行結果をGIN-forkに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[GIN-forkへの同期の失敗を解消する](../../conflict_helper.ipynb)を参照して、競合を解消してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

%store -r EXPERIMENT_TITLE
git_path = ['/home/jovyan/WORKFLOWS/EX-WORKFLOWS/util/required_rebuild_container.ipynb']
is_ok = utils.syncs_with_repo(git_path=git_path, gitannex_path=[], gitannex_files=[], message=EXPERIMENT_TITLE + '_実験環境再作成用セットアップ', get_paths=[])

## 7. フロー図を更新する
フロー図にこのタスクが実行済みであることを反映します。

In [ ]:
import os
os.chdir(os.environ['HOME'])

ex_path = 'WORKFLOWS/EX-WORKFLOWS/images/notebooks.diag'
re_path = 'WORKFLOWS/images/notebooks.diag'

with open('.gitignore', 'r') as f:
    has_re_diag = False
    has_ex_diag = False
    for line in f:
        if ex_path in line:
            has_ex_diag = True
        if re_path in line:
            has_re_diag = True
    
    if not has_re_diag:
        !echo $re_path >> ./.gitignore
    if not has_ex_diag:
        !echo $ex_path >> ./.gitignore


# フロー図に不要マークをつける
find = '"required_every_time"[fontsize = 10];'
replace = '"required_every_time"[numbered = 不要, fontsize = 10];'

with open(ex_path, 'r') as f:
    s = f.read()

with open(ex_path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

## 8. 不要なGIN-forkアクセストークンを削除する

In [ ]:
import os
import requests
os.chdir('/home/jovyan/WORKFLOWS')
from utils.token import token
from utils import display_util

origin_urls = !git config remote.origin.url
url = origin_urls[0]
try:
    # delete build token(only private)
    token.del_build_token_by_remote_origin_url(url)
except requests.exceptions.RequestException:
    display_util.display_err("現在、GIN-forkへの通信不良が発生しています。")

## 9. 実験フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックしてください。「実験フロートップページ」に遷移し、実験フローをご利用いただけます。  
「実験フロートップページ」のフロー図の「初期セットアップを行う」は実行いただく必要はありません。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../experiment.ipynb'>「実験フロートップページ」に遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))